You're given a two-dimensional array (a matrix) of potentially unequal height and width containing only ```0```s and ```1```s. Each ```0```represents land, and each ```1``` represents part of a river. A river consists of any number of ```1```s that are either horizontally or vertically adjacent (but not diagonally adjacent). The number of adjacent ```1```s forming a river determine its size.


Note that a river can twist. In other words, it doesn't have to be a straight vertical line or a straight horizontal line; it can be L-shaped, for example.


Write a function that returns an array of the sizes of all rivers represented in the input matrix. The sizes don't need to be in any particular order.

Example:

Input:
```
matrix = [
  [1, 0, 0, 1, 0],
  [1, 0, 1, 0, 0],
  [0, 0, 1, 0, 1],
  [1, 0, 1, 0, 1],
  [1, 0, 1, 1, 0],
]

```

Output:
```
[1, 2, 2, 2, 5] // The numbers could be ordered differently.

// The rivers can be clearly seen here:
// [
//   [1,  ,  , 1,  ],
//   [1,  , 1,  ,  ],
//   [ ,  , 1,  , 1],
//   [1,  , 1,  , 1],
//   [1,  , 1, 1,  ],
// ]
```

Reference: https://www.algoexpert.io/questions/River%20Sizes

In [1]:
"""
    Topic: Graph
    - Visit each node
        - if value is 1 => part of the river
                        => BFS / DFS
        - if value is 0 => STOP searching
        
Time Complexity: O(w * h); w - width of matrix; 
                    h - height of the matrix; n - number of elements in matrix
Space Complexity: O(w * h)
"""
def get_unvisited_neighbors(matrix, r, c, visit):
    # checking
    neighbors = []
    max_col_idx = len(matrix[0]) -1
    max_row_idx = len(matrix) -1
    # right cell: r, c+1
    if (c+1) <= max_col_idx and visit[r][c+1] == False:
        neighbors.append([r, c+1])
    # bottom cell: r+1, c
    if (r+1) <= max_row_idx and visit[r+1][c] == False:
        neighbors.append([r+1, c])
    # left cell: r, c-1
    if (c-1) >= 0 and visit[r][c-1] == False:
        neighbors.append([r, c-1])
    # top cell: r-1, c
    if (r-1) >=0 and visit[r-1][c] == False:
        neighbors.append([r-1, c])
    
    return neighbors

def traverse_node(matrix, row, col, visit, result):
    size = 0
    nodes = [[row, col]] # stack - DFS; use queue for BFS
    while len(nodes):
        [r, c] = nodes.pop()
        # visited before?
        if visit[r][c]:
            continue
        visit[r][c] = True #visit now
        
        if matrix[r][c] == 0:
            continue # not river, next "node"
        
        size = size + 1 # is a river
        
        # push top, left, right, bottom "node"
        unvisited_neighbors = get_unvisited_neighbors(matrix, r, c, visit)
        
        for n in unvisited_neighbors:
            nodes.append(n)
    
    if size !=0:
        result.append(size)
    return     

def river_sizes(matrix):
    visit = [[False for v in row] for row in matrix]
    result = []
    
    for row in range(len(matrix)):
        for col in range(len(matrix[row])):
            if visit[row][col]: # visited
                continue
            traverse_node(matrix, row, col, visit, result)
    
    return result

matrix = [
  [1, 0, 0, 1, 0],
  [1, 0, 1, 0, 0],
  [0, 0, 1, 0, 1],
  [1, 0, 1, 0, 1],
  [1, 0, 1, 1, 0],
]

river_sizes(matrix)

[2, 1, 5, 2, 2]

In [2]:
"""
    IDEA: Recursive Approach 
    boolean array with the same size as the matrix (all set with False)
    scan thru each element in matrix (Direction: from left to right; from top to bottom)
        if not visit before and the value is 1
            continue visit left, right and down position (helper function call) until it reach 0
            for each visit update the matrix at the same time
    stop until visit all the cell
            
Time Complexity - O(w * h); w - width of the matrix; h - height of matrix
Space Compexity - O(w * h); resursive call; 
                max call stack == max connected nodes, say d, 
                in worst situation, all nodes are connected, i.e. d = w * h
                => same as the size of visit matrix

"""

matrix = [
  [1, 0, 0, 1, 0],
  [1, 0, 1, 0, 0],
  [0, 0, 1, 0, 1],
  [1, 0, 1, 0, 1],
  [1, 0, 1, 1, 0],
]

def river_sizes_helper(matrix, r, c, visit):
    # update visit first
    visit[r][c] = True
    max_col_idx = len(matrix[r]) -1
    max_row_idx = len(matrix) -1
    # check if value is 1
    if matrix[r][c] == 1:
        size = 1
        # right cell: r, c+1
        if (c+1) <= max_col_idx and visit[r][c+1] == False:
            size = size + river_sizes_helper(matrix, r, c+1, visit)
        # bottom cell: r+1, c
        if (r+1) <= max_row_idx and visit[r+1][c] == False:
            size = size + river_sizes_helper(matrix, r+1, c, visit)
        # left cell: r, c-1
        if (c-1) >= 0 and visit[r][c-1] == False:
            size = size + river_sizes_helper(matrix, r, c-1, visit)
        # top cell: r-1, c
        if (r-1) >=0 and visit[r-1][c] == False:
            size = size + river_sizes_helper(matrix, r-1, c, visit)
        return size
    else: # move to next cell
        return 0
    
def river_sizes(matrix):
    # create visit matrix
    visit = [[False for v in row] for row in matrix]
    result = []
    
    r_next, c_next = 0, 0
    max_col_idx = len(matrix[0]) -1
    max_row_idx = len(matrix) -1
    all_visited = False
    while(not all_visited):
        size = river_sizes_helper(matrix, r_next, c_next, visit)
        if size > 0:
            result.append(size)
        while(visit[r_next][c_next]):
            # move to next column
            c_next = c_next+1
            if c_next > max_col_idx:
                c_next = 0
                r_next = r_next+1
            if r_next > max_row_idx:
                all_visited = True # all visited
                break
            if matrix[r_next][c_next] == 0: # no need to run
                visit[r_next][c_next] = True
    
    return result

print(river_sizes(matrix))

[2, 1, 5, 2, 2]
